In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install torch torchvision matplotlib

In [4]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2024-11-11 05:44:50--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2024-11-11 05:44:50--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  48.7MB/s    in 4.0s    

2024-11-11 05:44:54 (58.5 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [6]:
!unzip -q tiny-imagenet-200.zip

replace tiny-imagenet-200/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
import torch
from torchvision import models, datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model loading function
def load_model():
    model = models.resnet50(pretrained=True)  # Replace with desired architecture
    model.fc = nn.Linear(model.fc.in_features, 200)  # Adjust output layer for 200 classes
    model.to(device)
    return model

# Initialize the model, loss, and optimizer
model = load_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data transformations and loaders
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Load training and validation data
train_data = datasets.ImageFolder(root='tiny-imagenet-200/train', transform=transform)
val_data = datasets.ImageFolder(root='tiny-imagenet-200/val', transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 157MB/s]


Epoch 1/5, Loss: 3.697678081514892
Epoch 2/5, Loss: 2.722296950379321
Epoch 3/5, Loss: 2.270711747561215
Epoch 4/5, Loss: 1.871500079477741
Epoch 5/5, Loss: 1.478528464443014
Validation Accuracy: 0.58%


In [8]:
# Define a path for the saved model
model_path = "resnet50_tiny_imagenet.pth"

# Save the model's state_dict
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


Model saved to resnet50_tiny_imagenet.pth


In [9]:
from google.colab import files

# Download the saved model file
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>